In [1]:
import torch
from sklearn.datasets import load_diabetes

# Linear Regression

 Load sample dataset from sklearn dataset for regression problem

In [2]:
data = load_diabetes()

In [3]:
X, Y = data.data, data.target

In [4]:
data.feature_names

['age', 'sex', 'bmi', 'bp', 's1', 's2', 's3', 's4', 's5', 's6']

In [5]:
data.DESCR

'.. _diabetes_dataset:\n\nDiabetes dataset\n----------------\n\nTen baseline variables, age, sex, body mass index, average blood\npressure, and six blood serum measurements were obtained for each of n =\n442 diabetes patients, as well as the response of interest, a\nquantitative measure of disease progression one year after baseline.\n\n**Data Set Characteristics:**\n\n  :Number of Instances: 442\n\n  :Number of Attributes: First 10 columns are numeric predictive values\n\n  :Target: Column 11 is a quantitative measure of disease progression one year after baseline\n\n  :Attribute Information:\n      - age     age in years\n      - sex\n      - bmi     body mass index\n      - bp      average blood pressure\n      - s1      tc, total serum cholesterol\n      - s2      ldl, low-density lipoproteins\n      - s3      hdl, high-density lipoproteins\n      - s4      tch, total cholesterol / HDL\n      - s5      ltg, possibly log of serum triglycerides level\n      - s6      glu, blood sugar

In [6]:
type(X), type(Y)

(numpy.ndarray, numpy.ndarray)

In [7]:
X = torch.tensor(X).double()
Y = torch.tensor(Y).double()

In [8]:
X = X.cuda()
Y = Y.cuda()

In [9]:
type(X), type(Y)

(torch.Tensor, torch.Tensor)

In [10]:
X.shape, Y.shape

(torch.Size([442, 10]), torch.Size([442]))

## Linear Regression Equation
y = w1\*age + w2\*sex + w3\*bmi + w4\*bp + w5\*s1 + w6\*s2 + w7\*s3 + w8\*s4 + w9\*s5 + w10\*s6 + b0

Initializing weights and bias

torch.rand creates random number ranging from 0 to 1 in normal distribution

In [11]:
w = torch.randn(X.shape[1], requires_grad=True, device="cuda").double()
b = torch.randn([],requires_grad=True, device='cuda').double()

In [12]:
print(w,b)

tensor([ 0.7027, -1.3396, -0.9846, -0.7423,  0.2567,  1.3039, -0.5645,  1.4688,
        -0.2451, -0.2509], device='cuda:0', dtype=torch.float64,
       grad_fn=<ToCopyBackward0>) tensor(0.2594, device='cuda:0', dtype=torch.float64, grad_fn=<ToCopyBackward0>)


### Important to retain grad else you will get error when computing gradient

In [13]:
w.retain_grad()
b.retain_grad()

In [14]:
w.dtype

torch.float64

## Creating Dataset for Linear regression using Pytorch

In [15]:
from torch.utils.data import TensorDataset
dataset = TensorDataset(X, Y)

In [16]:
from torch.utils.data import DataLoader

batch_size = 3
train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [17]:
def model(X):
    return X @ w.t() + b

In [18]:
def mse_loss(predictions, targets):
    difference = predictions - targets
    return torch.sum(difference * difference)/ difference.numel()

In [19]:
for x,y in train_loader:
    preds = model(x)
    print("Prediction is :n",preds)
    print("nActual targets is :n",y)
    print("nLoss is: ",mse_loss(preds, y))
    break

Prediction is :n tensor([0.3592, 0.3657, 0.1548], device='cuda:0', dtype=torch.float64,
       grad_fn=<AddBackward0>)
nActual targets is :n tensor([162.,  93.,  85.], device='cuda:0', dtype=torch.float64)
nLoss is:  tensor(13969.1850, device='cuda:0', dtype=torch.float64,
       grad_fn=<DivBackward0>)


In [24]:
epochs = 2000
for i in range(epochs):
    # Iterate through training dataloader
    for x,y in train_loader:
        preds = model(x)
        # Get the loss and perform backpropagation
        loss = mse_loss(preds, y)
        loss.backward()
        
        # Update the weights
        with torch.no_grad():
            w -= w.grad *1e-3
            b -= b.grad *1e-3
            # Set the gradients to zero
            w.grad.zero_()
            b.grad.zero_()
    print(f"Epoch {i}/{epochs}: Loss: {loss}")

Epoch 0/2000: Loss: 9912.856800291189
Epoch 1/2000: Loss: 10607.969308112883
Epoch 2/2000: Loss: 1751.7805528924396
Epoch 3/2000: Loss: 7.71044895333494
Epoch 4/2000: Loss: 1901.642143319909
Epoch 5/2000: Loss: 2096.154221380483
Epoch 6/2000: Loss: 6027.520448306998
Epoch 7/2000: Loss: 340.8850973410045
Epoch 8/2000: Loss: 675.9487496565556
Epoch 9/2000: Loss: 3126.495663580309
Epoch 10/2000: Loss: 7597.289376515694
Epoch 11/2000: Loss: 208.02073176554742
Epoch 12/2000: Loss: 2397.5000503766078
Epoch 13/2000: Loss: 5591.079658069499
Epoch 14/2000: Loss: 5594.955686479456
Epoch 15/2000: Loss: 109.22394028120635
Epoch 16/2000: Loss: 5341.307866983676
Epoch 17/2000: Loss: 8509.812698283566
Epoch 18/2000: Loss: 22.815573348206033
Epoch 19/2000: Loss: 237.9503262398341
Epoch 20/2000: Loss: 5984.870718139249
Epoch 21/2000: Loss: 5087.8471547889285
Epoch 22/2000: Loss: 8071.509795699243
Epoch 23/2000: Loss: 721.101523080894
Epoch 24/2000: Loss: 6679.394025320881
Epoch 25/2000: Loss: 3964.3907

Epoch 206/2000: Loss: 237.70787181462143
Epoch 207/2000: Loss: 1447.6434465989219
Epoch 208/2000: Loss: 5945.122216523431
Epoch 209/2000: Loss: 696.5734290205767
Epoch 210/2000: Loss: 3772.0703898518605
Epoch 211/2000: Loss: 12484.06186500857
Epoch 212/2000: Loss: 11641.857876264201
Epoch 213/2000: Loss: 88.01037913540402
Epoch 214/2000: Loss: 1157.0369743926262
Epoch 215/2000: Loss: 707.4960425281223
Epoch 216/2000: Loss: 2138.145630618588
Epoch 217/2000: Loss: 7999.047499023807
Epoch 218/2000: Loss: 246.03922796131826
Epoch 219/2000: Loss: 1582.0332896835096
Epoch 220/2000: Loss: 3195.884731723247
Epoch 221/2000: Loss: 7163.142559261162
Epoch 222/2000: Loss: 5820.867950661232
Epoch 223/2000: Loss: 9376.169231280059
Epoch 224/2000: Loss: 9321.462302873095
Epoch 225/2000: Loss: 790.6291517040684
Epoch 226/2000: Loss: 509.661995181723
Epoch 227/2000: Loss: 15830.309273992085
Epoch 228/2000: Loss: 2053.698720163419
Epoch 229/2000: Loss: 695.7081802034035
Epoch 230/2000: Loss: 6235.149881

Epoch 409/2000: Loss: 31.108406455310057
Epoch 410/2000: Loss: 1272.6644067801128
Epoch 411/2000: Loss: 1539.4828372950828
Epoch 412/2000: Loss: 0.045740826553301694
Epoch 413/2000: Loss: 4639.810535283507
Epoch 414/2000: Loss: 6012.758207834038
Epoch 415/2000: Loss: 0.017017665979962352
Epoch 416/2000: Loss: 6203.963317762036
Epoch 417/2000: Loss: 10.889734081579773
Epoch 418/2000: Loss: 11824.334517871086
Epoch 419/2000: Loss: 862.8502929505418
Epoch 420/2000: Loss: 3635.496307110545
Epoch 421/2000: Loss: 5438.893436473248
Epoch 422/2000: Loss: 2145.041881582488
Epoch 423/2000: Loss: 5439.829313849653
Epoch 424/2000: Loss: 4181.634391750686
Epoch 425/2000: Loss: 1140.7997770577037
Epoch 426/2000: Loss: 2106.2224255693955
Epoch 427/2000: Loss: 5501.649944772037
Epoch 428/2000: Loss: 6231.735583050178
Epoch 429/2000: Loss: 3693.0743776114864
Epoch 430/2000: Loss: 1851.1601766461652
Epoch 431/2000: Loss: 4199.101302257453
Epoch 432/2000: Loss: 10148.731989017248
Epoch 433/2000: Loss: 59

Epoch 612/2000: Loss: 430.32091562747576
Epoch 613/2000: Loss: 328.7790498641485
Epoch 614/2000: Loss: 1.7302722101969672
Epoch 615/2000: Loss: 4062.739445583496
Epoch 616/2000: Loss: 5367.371411824311
Epoch 617/2000: Loss: 5717.073989673903
Epoch 618/2000: Loss: 9673.468680928923
Epoch 619/2000: Loss: 502.99420344372635
Epoch 620/2000: Loss: 1082.5854641373635
Epoch 621/2000: Loss: 10082.671833680373
Epoch 622/2000: Loss: 1295.7562211168015
Epoch 623/2000: Loss: 1075.2755528744908
Epoch 624/2000: Loss: 2512.196316186304
Epoch 625/2000: Loss: 3943.6636518726264
Epoch 626/2000: Loss: 6.0254545178138015
Epoch 627/2000: Loss: 3522.324263507398
Epoch 628/2000: Loss: 4282.393213499821
Epoch 629/2000: Loss: 1686.011562396471
Epoch 630/2000: Loss: 51.29121827180933
Epoch 631/2000: Loss: 475.9408985141454
Epoch 632/2000: Loss: 365.61554488445125
Epoch 633/2000: Loss: 432.2710262761342
Epoch 634/2000: Loss: 1675.450456612012
Epoch 635/2000: Loss: 843.4110418327574
Epoch 636/2000: Loss: 0.130713

Epoch 815/2000: Loss: 1708.536572972608
Epoch 816/2000: Loss: 76.05051699239442
Epoch 817/2000: Loss: 1390.3705099423635
Epoch 818/2000: Loss: 7315.80403177195
Epoch 819/2000: Loss: 6452.309839647265
Epoch 820/2000: Loss: 275.12870371660915
Epoch 821/2000: Loss: 6124.377640161921
Epoch 822/2000: Loss: 132.99074134391563
Epoch 823/2000: Loss: 52.16023356177995
Epoch 824/2000: Loss: 11687.917832122299
Epoch 825/2000: Loss: 2668.9734465269225
Epoch 826/2000: Loss: 181.17149365170116
Epoch 827/2000: Loss: 3868.68571222856
Epoch 828/2000: Loss: 2233.813561184973
Epoch 829/2000: Loss: 587.8116854413507
Epoch 830/2000: Loss: 14002.74843553615
Epoch 831/2000: Loss: 2488.0217360144907
Epoch 832/2000: Loss: 6428.267245815345
Epoch 833/2000: Loss: 9347.238736101917
Epoch 834/2000: Loss: 534.2312432270252
Epoch 835/2000: Loss: 407.1919404830399
Epoch 836/2000: Loss: 364.5995615724675
Epoch 837/2000: Loss: 2004.3431460684642
Epoch 838/2000: Loss: 3614.6539279436315
Epoch 839/2000: Loss: 23.69421333

Epoch 1018/2000: Loss: 3892.6092487347964
Epoch 1019/2000: Loss: 374.063761297241
Epoch 1020/2000: Loss: 1033.0694478925004
Epoch 1021/2000: Loss: 2.9337207678636767
Epoch 1022/2000: Loss: 1424.3793123104685
Epoch 1023/2000: Loss: 12.044155899607958
Epoch 1024/2000: Loss: 7554.887198344576
Epoch 1025/2000: Loss: 2203.3619251179343
Epoch 1026/2000: Loss: 5052.725366661392
Epoch 1027/2000: Loss: 4306.095214610945
Epoch 1028/2000: Loss: 7602.925633871194
Epoch 1029/2000: Loss: 46.86755978624876
Epoch 1030/2000: Loss: 263.90427266565996
Epoch 1031/2000: Loss: 1018.1796259439016
Epoch 1032/2000: Loss: 2466.734784200204
Epoch 1033/2000: Loss: 538.7019417277137
Epoch 1034/2000: Loss: 1897.24748155942
Epoch 1035/2000: Loss: 317.8176930739779
Epoch 1036/2000: Loss: 11.590615803982251
Epoch 1037/2000: Loss: 3860.765027010301
Epoch 1038/2000: Loss: 1257.5709285327343
Epoch 1039/2000: Loss: 2480.6531925650283
Epoch 1040/2000: Loss: 319.477327778891
Epoch 1041/2000: Loss: 2428.4145737864956
Epoch 1

Epoch 1216/2000: Loss: 1552.0247066473817
Epoch 1217/2000: Loss: 7588.922337270383
Epoch 1218/2000: Loss: 2917.950885217706
Epoch 1219/2000: Loss: 238.42421092540235
Epoch 1220/2000: Loss: 2274.388051130642
Epoch 1221/2000: Loss: 114.31615166672903
Epoch 1222/2000: Loss: 8848.82440056928
Epoch 1223/2000: Loss: 7810.647070386852
Epoch 1224/2000: Loss: 493.71159724177556
Epoch 1225/2000: Loss: 280.8813255106966
Epoch 1226/2000: Loss: 37.19325031960952
Epoch 1227/2000: Loss: 8.489891402303297
Epoch 1228/2000: Loss: 2201.950343174718
Epoch 1229/2000: Loss: 6993.729310397294
Epoch 1230/2000: Loss: 2495.4255075164842
Epoch 1231/2000: Loss: 25139.90265942035
Epoch 1232/2000: Loss: 6136.850971196696
Epoch 1233/2000: Loss: 592.5123115823109
Epoch 1234/2000: Loss: 11.331224286414404
Epoch 1235/2000: Loss: 48.63642358462711
Epoch 1236/2000: Loss: 2722.794068833643
Epoch 1237/2000: Loss: 8320.970133337278
Epoch 1238/2000: Loss: 9343.154304020662
Epoch 1239/2000: Loss: 55.97326572510598
Epoch 1240/

Epoch 1414/2000: Loss: 443.49453130865163
Epoch 1415/2000: Loss: 468.0905609333301
Epoch 1416/2000: Loss: 10761.832349348711
Epoch 1417/2000: Loss: 1595.4089719593287
Epoch 1418/2000: Loss: 432.98370487053626
Epoch 1419/2000: Loss: 9052.872919067082
Epoch 1420/2000: Loss: 980.9724028678777
Epoch 1421/2000: Loss: 2019.0345455287
Epoch 1422/2000: Loss: 514.4901862945907
Epoch 1423/2000: Loss: 593.2417963090056
Epoch 1424/2000: Loss: 422.054092840221
Epoch 1425/2000: Loss: 3158.4804108161316
Epoch 1426/2000: Loss: 2808.7482949720206
Epoch 1427/2000: Loss: 2671.3519192293174
Epoch 1428/2000: Loss: 3551.10535405122
Epoch 1429/2000: Loss: 6304.219203253495
Epoch 1430/2000: Loss: 8021.264807516494
Epoch 1431/2000: Loss: 4771.964879478527
Epoch 1432/2000: Loss: 8183.751917349983
Epoch 1433/2000: Loss: 9.627715465936097
Epoch 1434/2000: Loss: 0.12043200662408196
Epoch 1435/2000: Loss: 302.08635511724225
Epoch 1436/2000: Loss: 2275.953116467199
Epoch 1437/2000: Loss: 767.464645862066
Epoch 1438/

Epoch 1613/2000: Loss: 2365.6143863537886
Epoch 1614/2000: Loss: 506.3108015958475
Epoch 1615/2000: Loss: 0.5551900560299867
Epoch 1616/2000: Loss: 10637.49189352454
Epoch 1617/2000: Loss: 51.79993614332355
Epoch 1618/2000: Loss: 3692.884697910741
Epoch 1619/2000: Loss: 348.85851115873675
Epoch 1620/2000: Loss: 47.728838725352574
Epoch 1621/2000: Loss: 14129.631306961008
Epoch 1622/2000: Loss: 119.23632452357896
Epoch 1623/2000: Loss: 851.3331254802359
Epoch 1624/2000: Loss: 7567.591338738739
Epoch 1625/2000: Loss: 2140.1635475088215
Epoch 1626/2000: Loss: 689.2344459501318
Epoch 1627/2000: Loss: 1563.6176111994719
Epoch 1628/2000: Loss: 10271.202135874139
Epoch 1629/2000: Loss: 248.8548454406955
Epoch 1630/2000: Loss: 955.6440378309783
Epoch 1631/2000: Loss: 363.2200463541338
Epoch 1632/2000: Loss: 5050.435983801286
Epoch 1633/2000: Loss: 10116.663509735508
Epoch 1634/2000: Loss: 6.850095685775322
Epoch 1635/2000: Loss: 2422.2845431588344
Epoch 1636/2000: Loss: 853.538590271629
Epoch 

Epoch 1812/2000: Loss: 2938.962037756097
Epoch 1813/2000: Loss: 2584.7338939600354
Epoch 1814/2000: Loss: 1459.742816604619
Epoch 1815/2000: Loss: 1377.4160487955878
Epoch 1816/2000: Loss: 346.6608565853203
Epoch 1817/2000: Loss: 2623.5735159234073
Epoch 1818/2000: Loss: 135.7482597044932
Epoch 1819/2000: Loss: 4597.106335463284
Epoch 1820/2000: Loss: 3879.797151528778
Epoch 1821/2000: Loss: 3290.618639940617
Epoch 1822/2000: Loss: 10118.980719655818
Epoch 1823/2000: Loss: 1185.0640139214672
Epoch 1824/2000: Loss: 2783.847293168772
Epoch 1825/2000: Loss: 3877.544335058801
Epoch 1826/2000: Loss: 18.604811783735656
Epoch 1827/2000: Loss: 500.01881387340035
Epoch 1828/2000: Loss: 3751.748737917285
Epoch 1829/2000: Loss: 10782.207091614111
Epoch 1830/2000: Loss: 9036.651059963619
Epoch 1831/2000: Loss: 391.62547420265554
Epoch 1832/2000: Loss: 2062.834399310544
Epoch 1833/2000: Loss: 12864.460693869267
Epoch 1834/2000: Loss: 603.0741919075766
Epoch 1835/2000: Loss: 507.08095822436104
Epoch

In [27]:
for x,y in train_loader:
    preds = model(x)
    print("Prediction is :n",preds)
    print("Actual targets is :n",y)
    break

Prediction is :n tensor([174.8609, 140.5130, 220.0868], device='cuda:0', dtype=torch.float64,
       grad_fn=<AddBackward0>)
Actual targets is :n tensor([171., 182., 128.], device='cuda:0', dtype=torch.float64)


In [26]:
w

tensor([  10.4402, -186.7588,  475.7002,  297.3637,  -37.4102, -100.0639,
        -206.1816,  130.4304,  396.5597,  122.6037], device='cuda:0',
       dtype=torch.float64, grad_fn=<ToCopyBackward0>)

# Using built-in Classes

In [42]:
X = X.float()
Y = Y.float()

In [43]:
import torch.nn as nn

In [49]:
model1 = nn.Linear(10,1)

In [50]:
model1 = model1.cuda()

In [51]:
print(model1.weight, model1.bias)

Parameter containing:
tensor([[-0.2889,  0.1869,  0.0107, -0.0569, -0.0831,  0.1190, -0.1590,  0.1502,
          0.1560, -0.1879]], device='cuda:0', requires_grad=True) Parameter containing:
tensor([-0.0793], device='cuda:0', requires_grad=True)


In [52]:
mse_loss = nn.functional.mse_loss

In [53]:
loss = mse_loss(model1(X),Y)
print(loss)

tensor(29098.6152, device='cuda:0', grad_fn=<MseLossBackward0>)


C:\Users\accma\AppData\Local\Temp\ipykernel_9752\2577316876.py:1: UserWarning: Using a target size (torch.Size([442])) that is different to the input size (torch.Size([442, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = mse_loss(model1(X),Y)


In [54]:
opt = torch.optim.SGD(model1.parameters(), 1e-3)

In [61]:
def fit(model, loss_fn, opt, train_loader, epochs):
    for i in range(epochs):
        # Iterate through training dataloader
        for x,y in train_loader:
            preds = model(x.float())
            # Get the loss and perform backpropagation
            loss = loss_fn(preds, y.float())
            loss.backward()

            opt.step()
            opt.zero_grad()
        print(f"Epoch {i}/{epochs}: Loss: {loss}")
    return model

In [62]:
model1 = fit(model1, mse_loss, opt, train_loader, 1000)

C:\Users\accma\AppData\Local\Temp\ipykernel_9752\1938789256.py:7: UserWarning: Using a target size (torch.Size([3])) that is different to the input size (torch.Size([3, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = loss_fn(preds, y.float())
C:\Users\accma\AppData\Local\Temp\ipykernel_9752\1938789256.py:7: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = loss_fn(preds, y.float())


Epoch 0/1000: Loss: 11742.6796875
Epoch 1/1000: Loss: 10336.1396484375
Epoch 2/1000: Loss: 97.19794464111328
Epoch 3/1000: Loss: 64.92765045166016
Epoch 4/1000: Loss: 13304.603515625
Epoch 5/1000: Loss: 6282.30859375
Epoch 6/1000: Loss: 67.12613677978516
Epoch 7/1000: Loss: 63.48590087890625
Epoch 8/1000: Loss: 14287.078125
Epoch 9/1000: Loss: 5462.71337890625
Epoch 10/1000: Loss: 1070.461669921875
Epoch 11/1000: Loss: 835.6842041015625
Epoch 12/1000: Loss: 5682.79638671875
Epoch 13/1000: Loss: 979.2833251953125
Epoch 14/1000: Loss: 930.8848876953125
Epoch 15/1000: Loss: 6311.99658203125
Epoch 16/1000: Loss: 1744.5267333984375
Epoch 17/1000: Loss: 1731.529296875
Epoch 18/1000: Loss: 15011.91015625
Epoch 19/1000: Loss: 1506.6151123046875
Epoch 20/1000: Loss: 11593.068359375
Epoch 21/1000: Loss: 7938.67529296875
Epoch 22/1000: Loss: 6221.19091796875
Epoch 23/1000: Loss: 4468.48486328125
Epoch 24/1000: Loss: 11338.7626953125
Epoch 25/1000: Loss: 203.2960968017578
Epoch 26/1000: Loss: 460.

Epoch 214/1000: Loss: 1131.108642578125
Epoch 215/1000: Loss: 572.1488037109375
Epoch 216/1000: Loss: 1961.8494873046875
Epoch 217/1000: Loss: 6547.2978515625
Epoch 218/1000: Loss: 103.90332794189453
Epoch 219/1000: Loss: 1035.6668701171875
Epoch 220/1000: Loss: 2065.612060546875
Epoch 221/1000: Loss: 4528.95556640625
Epoch 222/1000: Loss: 1082.565185546875
Epoch 223/1000: Loss: 2872.90673828125
Epoch 224/1000: Loss: 1952.80078125
Epoch 225/1000: Loss: 692.6937866210938
Epoch 226/1000: Loss: 18804.224609375
Epoch 227/1000: Loss: 4120.21142578125
Epoch 228/1000: Loss: 13438.0224609375
Epoch 229/1000: Loss: 10055.5458984375
Epoch 230/1000: Loss: 8552.451171875
Epoch 231/1000: Loss: 29630.865234375
Epoch 232/1000: Loss: 4355.33349609375
Epoch 233/1000: Loss: 1964.3916015625
Epoch 234/1000: Loss: 1779.8482666015625
Epoch 235/1000: Loss: 337.62493896484375
Epoch 236/1000: Loss: 2509.823974609375
Epoch 237/1000: Loss: 17483.650390625
Epoch 238/1000: Loss: 1544.2779541015625
Epoch 239/1000: L

Epoch 426/1000: Loss: 2585.212646484375
Epoch 427/1000: Loss: 13072.5146484375
Epoch 428/1000: Loss: 0.4836292266845703
Epoch 429/1000: Loss: 32.97061538696289
Epoch 430/1000: Loss: 3023.82861328125
Epoch 431/1000: Loss: 2906.35107421875
Epoch 432/1000: Loss: 17288.125
Epoch 433/1000: Loss: 7868.05224609375
Epoch 434/1000: Loss: 31833.41796875
Epoch 435/1000: Loss: 10694.875
Epoch 436/1000: Loss: 23571.8359375
Epoch 437/1000: Loss: 4029.551025390625
Epoch 438/1000: Loss: 5624.35205078125
Epoch 439/1000: Loss: 8555.9814453125
Epoch 440/1000: Loss: 2909.90576171875
Epoch 441/1000: Loss: 1410.0806884765625
Epoch 442/1000: Loss: 7142.45751953125
Epoch 443/1000: Loss: 1997.9901123046875
Epoch 444/1000: Loss: 3889.768798828125
Epoch 445/1000: Loss: 23637.48046875
Epoch 446/1000: Loss: 844.3309326171875
Epoch 447/1000: Loss: 7543.1884765625
Epoch 448/1000: Loss: 1258.214111328125
Epoch 449/1000: Loss: 13015.3291015625
Epoch 450/1000: Loss: 1298.58203125
Epoch 451/1000: Loss: 6151.02685546875


Epoch 639/1000: Loss: 26155.0078125
Epoch 640/1000: Loss: 4983.0576171875
Epoch 641/1000: Loss: 101.64735412597656
Epoch 642/1000: Loss: 24.82832908630371
Epoch 643/1000: Loss: 119.26235961914062
Epoch 644/1000: Loss: 289.3061828613281
Epoch 645/1000: Loss: 10305.294921875
Epoch 646/1000: Loss: 1466.1610107421875
Epoch 647/1000: Loss: 5.966128349304199
Epoch 648/1000: Loss: 1638.72265625
Epoch 649/1000: Loss: 5309.47119140625
Epoch 650/1000: Loss: 399.8486328125
Epoch 651/1000: Loss: 2.3707804679870605
Epoch 652/1000: Loss: 1295.09375
Epoch 653/1000: Loss: 7831.671875
Epoch 654/1000: Loss: 1509.16650390625
Epoch 655/1000: Loss: 4057.80419921875
Epoch 656/1000: Loss: 1825.53564453125
Epoch 657/1000: Loss: 5229.59423828125
Epoch 658/1000: Loss: 4774.62548828125
Epoch 659/1000: Loss: 2244.334228515625
Epoch 660/1000: Loss: 6355.09375
Epoch 661/1000: Loss: 375.9099426269531
Epoch 662/1000: Loss: 1505.988525390625
Epoch 663/1000: Loss: 7452.84033203125
Epoch 664/1000: Loss: 252.288940429687

Epoch 850/1000: Loss: 7195.94775390625
Epoch 851/1000: Loss: 48.15410232543945
Epoch 852/1000: Loss: 118.62000274658203
Epoch 853/1000: Loss: 2371.043212890625
Epoch 854/1000: Loss: 7228.81884765625
Epoch 855/1000: Loss: 2787.26513671875
Epoch 856/1000: Loss: 3411.5146484375
Epoch 857/1000: Loss: 97.07853698730469
Epoch 858/1000: Loss: 851.5396728515625
Epoch 859/1000: Loss: 3827.62451171875
Epoch 860/1000: Loss: 49.47453308105469
Epoch 861/1000: Loss: 35.45768356323242
Epoch 862/1000: Loss: 1768.7366943359375
Epoch 863/1000: Loss: 421.28350830078125
Epoch 864/1000: Loss: 7193.5380859375
Epoch 865/1000: Loss: 1193.4127197265625
Epoch 866/1000: Loss: 4571.0029296875
Epoch 867/1000: Loss: 8246.94921875
Epoch 868/1000: Loss: 325.5041198730469
Epoch 869/1000: Loss: 4212.37548828125
Epoch 870/1000: Loss: 8108.0029296875
Epoch 871/1000: Loss: 1630.2158203125
Epoch 872/1000: Loss: 2550.08056640625
Epoch 873/1000: Loss: 23676.984375
Epoch 874/1000: Loss: 11938.7705078125
Epoch 875/1000: Loss: 

In [63]:
preds = model1(X)

In [65]:
preds

tensor([[163.7674],
        [127.8626],
        [157.5688],
        [153.3948],
        [144.6713],
        [131.2345],
        [133.4283],
        [152.1910],
        [155.9860],
        [161.9551],
        [132.2560],
        [145.2506],
        [139.8369],
        [154.5648],
        [136.5008],
        [162.8504],
        [162.3097],
        [163.9188],
        [147.6383],
        [140.1754],
        [140.3699],
        [136.2724],
        [140.8574],
        [187.6870],
        [149.8828],
        [150.6387],
        [131.3309],
        [154.6169],
        [141.7642],
        [157.8701],
        [149.7124],
        [126.4130],
        [180.4675],
        [135.6933],
        [130.3717],
        [136.8946],
        [160.4161],
        [147.9964],
        [179.2487],
        [149.4770],
        [160.1076],
        [127.7067],
        [149.1584],
        [134.0711],
        [170.1986],
        [146.0235],
        [141.6082],
        [133.3095],
        [136.3832],
        [161.9853],


In [64]:
Y

tensor([151.,  75., 141., 206., 135.,  97., 138.,  63., 110., 310., 101.,  69.,
        179., 185., 118., 171., 166., 144.,  97., 168.,  68.,  49.,  68., 245.,
        184., 202., 137.,  85., 131., 283., 129.,  59., 341.,  87.,  65., 102.,
        265., 276., 252.,  90., 100.,  55.,  61.,  92., 259.,  53., 190., 142.,
         75., 142., 155., 225.,  59., 104., 182., 128.,  52.,  37., 170., 170.,
         61., 144.,  52., 128.,  71., 163., 150.,  97., 160., 178.,  48., 270.,
        202., 111.,  85.,  42., 170., 200., 252., 113., 143.,  51.,  52., 210.,
         65., 141.,  55., 134.,  42., 111.,  98., 164.,  48.,  96.,  90., 162.,
        150., 279.,  92.,  83., 128., 102., 302., 198.,  95.,  53., 134., 144.,
        232.,  81., 104.,  59., 246., 297., 258., 229., 275., 281., 179., 200.,
        200., 173., 180.,  84., 121., 161.,  99., 109., 115., 268., 274., 158.,
        107.,  83., 103., 272.,  85., 280., 336., 281., 118., 317., 235.,  60.,
        174., 259., 178., 128.,  96., 12